In [ ]:
import pysec
import pandas as pd


from bs4 import BeautifulSoup
import urllib2
import re

short_cik = '51143';
long_cik  = '0000051143';

tenq = pysec.util.edgar_utilies.get_acc_table({'cik': long_cik ,  'qk' : 'q'})
tenk = pysec.util.edgar_utilies.get_acc_table({'cik': short_cik , 'qk' : 'k'})

final_df = pd.concat([tenk,tenq])
final_df.reset_index(inplace=True)
final_df.drop("index",axis=1,inplace=True)

def secinfo (s,a):    return 'https://www.sec.gov/cgi-bin/viewer?action=view&cik=%s&accession_number=%s' % (s,a)
def sectableHTML(s,a,d):  return 'https://www.sec.gov/Archives/edgar/data/%s/%s/R%s.htm' % (s,a,d)
def sectableXML(s,a,d):   return 'https://www.sec.gov/Archives/edgar/data/%s/%s/R%s.xml' % (s,a,d)
def secxrbl (s,a,d):  return 'https://www.sec.gov/Archives/edgar/data/%s/%s/%s' % (s,a,d)
def secfilelist(s,a): return 'https://www.sec.gov/Archives/edgar/data/%s/%s' % (s,a)

In [ ]:
final_df

In [ ]:
import stupidxbrl
import urllib2
import re
XBRL_REGEX = re.compile('^[a-zA-Z]+-[0-9]{8}.xml$')

def builder(row):
    page = urllib2.urlopen(secfilelist(short_cik,pysec.util.edgar_utilies.remove_dashes_acc(row['Acc'])))
    soup = BeautifulSoup(page,'lxml')
    
    f = ''
    for a in soup.find_all('a'): 
        if XBRL_REGEX.search(a.text) is not None: 
            f = a.text; 
            break;
    
    s = stupidxbrl.stupidXBRL(urllib2.urlopen(secxrbl(short_cik,
                                                      pysec.util.edgar_utilies.remove_dashes_acc(row['Acc'])
                                                      ,f)))
    s.Build()
    return s

In [ ]:
final_df

In [ ]:
final_df['sxbrl'] = final_df.apply(builder,axis=1)

In [ ]:
# Really annoying, sec changed from XML to HTML between 2011 and 2012 which means need
# to write another parser to parse XML instead of this one... i'm steaming ATM

def gaapconversion(row):
    
    #page = urllib2.urlopen(secinfo(short_cik,
    #                               pysec.util.edgar_utilies.remove_dashes_acc(row['Acc'])))
    #soup = BeautifulSoup(page,"lxml")
    #
    #statements = {}
    #for i in soup.find_all(class_='xbrlviewer'):
    #    match = re.search("loadReport\(([0-9]+)\)",i.attrs['href'])
    #    if match is not None:
    #        statements[i.text] = match.group(1)
    #        
    #statement_data = {}
    #print statements
    
    #for key in statements:
    fff = 2
    data = {}
    curr_abstract = ''
    HTML = True
    page2=None
    try:
        page2 = urllib2.urlopen(sectableHTML(short_cik,
                                             pysec.util.edgar_utilies.remove_dashes_acc(row['Acc']),
                                             fff))
        
    except urllib2.HTTPError:
        page2 = urllib2.urlopen(sectableXML(short_cik,
                                             pysec.util.edgar_utilies.remove_dashes_acc(row['Acc']),
                                             fff-1)) 
        HTML = False
        pass
        
    if HTML :
        soup2 = BeautifulSoup(page2,"lxml")

        
        data = {}
        curr_abstract = ''
        for f in soup2.find_all(class_="pl"):
    
            fmatch = re.search("defref\_((us-gaap|ibm|dei)\_[a-zA-Z]+)",f.a['onclick']).group(1).split("_")

            if re.search("Abstract",fmatch[1]) is not None:
                curr_abstract = f.text.rstrip(":")
               
            if data.has_key(curr_abstract):
                data[curr_abstract][f.text] = fmatch
            else:
                data[curr_abstract] = {}
        
        
    
    else:
        soup2 = BeautifulSoup(page2,"lxml-xml")
        
        for raw in soup2.find("Rows").find_all("Row"):
            
            if re.search("Abstract",raw.ElementName.text) is not None:
                curr_abstract = raw.Label.text.rstrip(":")
                print curr_abstract
               
            if data.has_key(curr_abstract):
                data[curr_abstract][raw.Label.text] = raw.ElementName.text
            else:
                data[curr_abstract] = {}
    
    return data

In [ ]:
final_df['aaa'] = final_df.apply(gaapconversion,axis=1)

In [ ]:
c = 0;
for i in final_df.aaa: 
    if i is not None: 
        print final_df.iloc[c].sxbrl.gaap[i['Revenue']['Sales'][1]].keys()
    c += 1

In [ ]:
final_df.iloc[0].sxbrl.gaap

In [ ]:
f

In [ ]:
print secxrbl(short_cik,'000104746915001106',f)
s = stupidxbrl.stupidXBRL(urllib2.urlopen(secxrbl(short_cik,'000104746915001106',f)))

In [ ]:
final_df.iloc[0].aaa['Revenue']

In [ ]:
for i in final_df.aaa: 
    if i is not None: 
        if 'Revenue' not in i.keys():
            print i
    c += 1

In [ ]:
for i in final_df.index:
    print final_df.ix[i].conversion()
    print ''

In [ ]:
s.document.keys()